In [ ]:
from tkinter import *
from tkinter import ttk
from ttkthemes import ThemedTk
import joblib


Sex = 'Sex', 
fields = 'Age', 'BMI', 'Cigarettes per day', 'Sys BP', 'Dia BP', 'Heart Rate','Total Cholestrol'

check = 'Blood Pressure Medication', 'Prevalent Hypertension', 'Prevalent Stroke', 'Diabetes'


def fetch(entries):
    for entry in entries:
        field = entry[0]
        text  = entry[1].get()
        LR = joblib.load('LR.pkl') 
        print('%s: "%s"' % (field, text))
        LR.predict(text)
        


def makeform(root, fields):
    entries = []
    for field in Sex:
        row = Frame(root)
        lab = Label(row, width=20, text=field, anchor='w')
        ent = ttk.Combobox(row, values = ['Male', 'Female'])
        row.pack(side=TOP, fill=X, padx=5, pady=5)
        lab.pack(side=LEFT)
        ent.pack(side=RIGHT, expand=YES, fill=X)
        entries.append((field, ent))
        
    for field in fields:
        row = Frame(root)
        lab = Label(row, width=20, text=field, anchor='w')
        ent = Entry(row)
        row.pack(side=TOP, fill=X, padx=5, pady=5)
        lab.pack(side=LEFT)
        ent.pack(side=RIGHT, expand=YES, fill=X)
        entries.append((field, ent))
    
    for field in check:
        row = Frame(root)
        lab = Label(row, width=20, text=field, anchor='w')
        ent = BooleanVar() 
        chk = Checkbutton(row,  var=ent)  
        row.pack(side=TOP, fill=X, padx=5, pady=5 )
        lab.pack(side=LEFT)
        chk.pack(side=RIGHT, anchor='w')
        entries.append((field, ent)) 
          
    return entries

    
if __name__ == '__main__':
    root = ThemedTk(theme = 'arc')
    root.title('CHD predictor')
    root.resizable(False, False)
    ents = makeform(root, fields)
    root.bind(root, (lambda event, e=ents: fetch(e)))   
    b1 = Button(root, text='Show',
                  command=(lambda e=ents: fetch(e)))
    b1.pack(side=RIGHT, padx=5, pady=5)
    b2 = Button(root, text='Quit', command=root.quit)
    b2.pack(side=RIGHT, padx=5, pady=5)
    root.mainloop()

Sex: "Female"


Exception in Tkinter callback
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.7/tkinter/__init__.py", line 1705, in __call__
    return self.func(*args)
  File "<ipython-input-10-f4c0e1b09299>", line 62, in <lambda>
    command=(lambda e=ents: fetch(e)))
  File "<ipython-input-10-f4c0e1b09299>", line 18, in fetch
    LR.predict(text)
  File "/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/base.py", line 289, in predict
    scores = self.decision_function(X)
  File "/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/base.py", line 265, in decision_function
    X = check_array(X, accept_sparse='csr')
  File "/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py", line 514, in check_array
    "if it contains a single sample.".format(array))
ValueError: Expected 2D array, got scalar array instead:
array=Female.
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if 

In [8]:
import joblib

In [9]:
LR_from_joblib = joblib.load('LR.pkl')  
